In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from sklearn import metrics

In [2]:
cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
if current_folder != 'BFOR-516-Project':
    os.chdir('..')
    cwd = os.getcwd()
current_folder = cwd.split('\\')[-1]
print(current_folder)
cwd

models
BFOR-516-Project


'C:\\Users\\jacks\\Documents\\GitHub\\BFOR-516-Project'

In [3]:
data = pd.read_csv('data/datafm20.csv')

In [4]:
data

,Name,Position,Club,Division,Based,Nation,Height,Weight,Age,Preferred Foot,...,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc
0,Lionel Messi,"AM (RC), ST (C)",Barcelona,Spanish First Division,Spain (First Division),ARG,170,72,32,Left,...,18,2,2,10,19,19,19,7,2,18
1,Cristiano Ronaldo,"AM (RL), ST (C)",Juventus,Italian Serie A,Italy (Serie A),POR,185,83,34,Either,...,14,4,1,16,14,18,13,6,2,15
2,Kylian MbappÃ©,"AM (RL), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),FRA,178,73,20,Right,...,18,2,1,12,14,17,16,6,4,20
3,Manuel Neuer,GK,FC Bayern,Bundesliga,Germany (Bundesliga),GER,192,90,33,Either,...,20,12,17,14,14,19,15,8,14,13
4,Neymar,"M (L), AM (LC), ST (C)",Paris SG,Ligue 1 Conforama,France (Ligue 1 Conforama),BRA,175,68,27,Right,...,17,3,1,14,14,16,18,10,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144745,Yuan Jinxing,ST (C),Unknown,Unknown,China,CHN,169,81,41,Right Only,...,7,3,3,2,3,9,8,8,1,11
144746,Aizudin Abdul Rahman,M (C),BSRC,Bruneian Premier League,Brunei (Premier League),BRU,180,69,24,Right Only,...,4,1,3,1,3,4,9,14,2,9
144747,Berlin Jean-Gilles,ST (C),CS Monteuil,PremiÃ¨re Ligue de soccer du QuÃ©bec,Canada (PLSQ),HAI,180,79,32,Right Only,...,8,3,1,1,2,4,10,6,3,11
144748,Frederik van Hauen,M (C),Skovshoved,2. Division,Denmark (2. Division),DEN,195,85,19,Right Only,...,3,3,3,3,4,4,3,16,1,5


In [5]:
minimums = data.min()
maximums = data.max()
#minimums.to_csv('minimums.csv')
#maximums.to_csv('maximums.csv')

Value and wage have 0s so might have to use log for those two

In [6]:
data['Ant'].dtypes

dtype('int64')

In [30]:
int_list = []
pred_vars = []

for i in data.columns:
    if data[i].dtypes == 'int64':
        int_list.append(i)
        if i != 'CA' and i !='PA':
            pred_vars.append(i)

In [8]:
corr = data[int_list].corr()
corr.style.background_gradient(cmap='coolwarm')

,Height,Weight,Age,Value,Wage,CA,PA,Wor,Vis,Thr,Tec,Tea,Tck,Str,Sta,TRO,Ref,Pun,Pos,Pen,Pas,Pac,1v1,OtB,Nat,Mar,L Th,Lon,Ldr,Kic,Jum,Hea,Han,Fre,Fla,Fir,Fin,Ecc,Dri,Det,Dec,Cro,Cor,Cnt,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc
Height,1.000000,0.790270,0.048629,0.021120,0.024942,0.076890,0.059560,-0.141420,-0.192301,0.272584,-0.292641,-0.008698,-0.019366,0.360370,-0.105444,0.282027,0.303359,0.279315,0.229945,-0.067279,-0.137968,-0.232315,0.282575,-0.241078,0.017983,0.025868,-0.070726,-0.215612,0.054739,0.288263,0.769151,0.139777,0.303184,-0.074850,-0.236316,-0.297125,-0.178392,0.162173,-0.288349,0.020050,0.084630,-0.298818,-0.240485,0.127001,-0.028173,0.283253,0.290370,0.235186,0.092150,0.089000,-0.243586,0.032998,0.327978,-0.262820
Weight,0.790270,1.000000,0.274063,0.049281,0.058936,0.249639,0.161103,0.008743,-0.063505,0.249706,-0.158673,0.165080,0.051282,0.551687,0.062155,0.233436,0.250476,0.232060,0.326303,0.087870,-0.021370,-0.234372,0.254992,-0.092786,-0.041850,0.106106,0.063888,-0.071271,0.141025,0.248535,0.694555,0.231907,0.249714,0.056031,-0.138915,-0.191685,-0.069524,0.158504,-0.165969,0.053931,0.097234,-0.164049,-0.107743,0.277984,0.138192,0.254347,0.253600,0.348554,0.282753,0.256208,-0.215178,0.073557,0.275978,-0.271931
Age,0.048629,0.274063,1.000000,0.040546,0.073089,0.296342,0.092173,0.235134,0.173740,0.050242,0.127561,0.330482,0.086274,0.379263,0.315064,0.018469,0.006307,0.019915,0.243485,0.309101,0.156565,-0.145307,0.048038,0.196127,-0.004369,0.123241,0.232615,0.190034,0.303110,0.023570,0.151741,0.156826,0.009398,0.241828,0.145065,0.055901,0.115485,0.057289,0.107136,0.084557,0.012726,0.168790,0.172092,0.340873,0.310602,0.050252,0.033244,0.286951,0.335420,0.350986,-0.064017,0.107875,0.028374,-0.195395
Value,0.021120,0.049281,0.040546,1.000000,0.763292,0.326957,0.286218,0.178467,0.177202,0.004797,0.169782,0.208125,0.078166,0.166927,0.177811,-0.003040,-0.005287,-0.008685,0.100763,0.183799,0.176875,0.128165,0.005524,0.154641,0.076975,0.066055,0.104486,0.155860,0.069209,-0.001638,0.082608,0.097222,-0.008642,0.156182,0.136361,0.145076,0.129317,0.008712,0.151187,0.097741,0.105903,0.142231,0.133609,0.188301,0.215416,0.001132,-0.001887,0.163199,0.212499,0.205335,0.130989,0.071993,-0.005238,0.101920
Wage,0.024942,0.058936,0.073089,0.763292,1.000000,0.306466,0.270332,0.161506,0.164798,0.010082,0.158015,0.199219,0.071789,0.164677,0.164583,0.001683,0.000324,-0.004923,0.098836,0.180503,0.165641,0.107111,0.012312,0.142671,0.068988,0.060405,0.105603,0.147875,0.075424,0.002603,0.084804,0.097449,-0.002833,0.153742,0.126871,0.133886,0.121339,0.015235,0.136399,0.092806,0.099960,0.131787,0.127897,0.181935,0.207909,0.007815,0.004356,0.159950,0.206039,0.200642,0.113288,0.066046,0.000205,0.077994
CA,0.076890,0.249639,0.296342,0.326957,0.306466,1.000000,0.859521,0.581409,0.561502,0.073998,0.531869,0.709459,0.298931,0.623386,0.567241,-0.009060,0.001791,-0.012333,0.475408,0.543884,0.601253,0.269658,0.060882,0.492931,0.079361,0.317486,0.479528,0.521084,0.223545,0.030932,0.330086,0.395560,-0.002178,0.560957,0.349731,0.430831,0.379396,0.048916,0.458322,0.184661,0.290213,0.486116,0.524601,0.663173,0.662044,0.045140,0.029239,0.555406,0.723519,0.687894,0.302087,0.140309,0.010254,0.188974
PA,0.059560,0.161103,0.092173,0.286218,0.270332,0.859521,1.000000,0.485905,0.509560,0.067196,0.490375,0.602679,0.264767,0.509762,0.433355,-0.004851,0.006144,-0.009872,0.390118,0.469681,0.552309,0.215659,0.057120,0.414957,0.070558,0.261362,0.398674,0.464093,0.141624,0.032341,0.277155,0.345306,0.002223,0.492098,0.317330,0.401876,0.339272,0.040638,0.413734,0.175693,0.264140,0.424214,0.462811,0.548684,0.555864,0.036999,0.026156,0.488480,0.610402,0.573239,0.241473,0.115911,0.008599,0.150812
Wor,-0.141420,0.008743,0.235134,0.178467,0.161506,0.581409,0.485905,1.000000,0.603221,-0.180702,0.574603,0.749532,0.399150,0.440133,0.636274,-0.274119,-0.295734,-0.273137,0.276026,0.423923,0.564028,0.164380,-0.204856,0.468019,0.060452,0.366834,0.471721,0.568617,0.176166,-0.251316,0.

In [9]:
corr2 = corr[['CA', 'PA']]
corr2.style.background_gradient(cmap='coolwarm')

,CA,PA
Height,0.076890,0.059560
Weight,0.249639,0.161103
Age,0.296342,0.092173
Value,0.326957,0.286218
Wage,0.306466,0.270332
CA,1.000000,0.859521
PA,0.859521,1.000000
Wor,0.581409,0.485905
Vis,0.561502,0.509560
Thr,0.073998,0.067196


In [31]:
from sklearn.model_selection import train_test_split

# use index-based sampling since we have time series data
train, test = train_test_split(data, test_size=0.5, shuffle=True)

In [32]:
print(len(train))
# train = train.dropna(subset=pred_vars)
print(len(test))

72375
72375


In [33]:
from sklearn.ensemble import RandomForestRegressor

rf_reg_PA = RandomForestRegressor()

rf_reg_PA.fit(train[pred_vars], train['PA'])

rf_reg_CA = RandomForestRegressor()

rf_reg_CA.fit(train[pred_vars], train['CA'])

RandomForestRegressor()

In [43]:
#About an hour and half
#Took forever and RMSE was
from sklearn.svm import SVR

svr_reg_PA = SVR()

svr_reg_PA.fit(train[pred_vars], train['PA'])

svr_reg_CA = SVR()

svr_reg_CA.fit(train[pred_vars], train['CA'])

SVR()

In [34]:
#MLP preformed horribly
from sklearn.neural_network import MLPRegressor

mlp_reg_CA = MLPRegressor()

mlp_reg_CA.fit(train[pred_vars], train['CA'])

mlp_reg_PA = MLPRegressor()

mlp_reg_PA.fit(train[pred_vars], train['PA'])

MLPRegressor()

In [35]:
#Second best so farto Random Forest
from sklearn.neighbors import KNeighborsClassifier

kn_reg_PA = KNeighborsClassifier()
kn_reg_PA.fit(train[pred_vars], train['PA'])

kn_reg_CA = KNeighborsClassifier()
kn_reg_CA.fit(train[pred_vars], train['CA'])

KNeighborsClassifier()

In [36]:
reg_multi_PA = sm.OLS(train['PA'], train[pred_vars], hasconst=False).fit()
reg_multi_PA.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     PA   R-squared (uncentered):                   0.981
Model:                            OLS   Adj. R-squared (uncentered):              0.981
Method:                 Least Squares   F-statistic:                          7.034e+04
Date:                Sun, 21 Nov 2021   Prob (F-statistic):                        0.00
Time:                        20:15:55   Log-Likelihood:                     -2.9162e+05
No. Observations:               72375   AIC:                                  5.833e+05
Df Residuals:                   72323   BIC:                                  5.838e+05
Df Model:                          52                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Height        -0.0902      0.006    -15.451      0.000      -0.102      -0.079
Weight        -0.0553      0.013     -4.375      0.000      -0.080      -0.031
Age           -0.8341      0.011    -74.898      0.000      -0.856      -0.812
Value       1.393e-07   3.56e-08      3.909      0.000    6.94e-08    2.09e-07
Wage           0.0001   8.69e-06     13.769      0.000       0.000       0.000
Wor           -0.0801      0.031     -2.609      0.009      -0.140      -0.020
Vis            0.2988      0.027     10.999      0.000       0.246       0.352
Thr            0.3802      0.045      8.470      0.000       0.292       0.468
Tec            0.6503      0.030     21.391      0.000       0.591       0.710
Tea           -0.2639      0.037     -7.180      0.000      -0.336      -0.192
Tck            0.5085      0.026     19.378      0.000       0.457       0.560
Str            0.5959      0.027     21.723      0.000       0.542       0.650
Sta            0.1712      0.023      7.444      0.000       0.126       0.216
TRO            0.0582      0.037      1.558      0.119      -0.015       0.131
Ref            0.5457      0.043     12.684      0.000       0.461       0.630
Pun           -0.0154      0.036     -0.426      0.670      -0.086       0.055
Pos            0.6113      0.027     22.246      0.000       0.557       0.665
Pen            0.2532      0.023     11.119      0.000       0.209       0.298
Pas            0.8591      0.028     30.941      0.000       0.805       0.914
Pac            0.4921      0.037     13.189      0.000       0.419       0.565
1v1            0.4977      0.045     10.961      0.000       0.409       0.587
OtB            0.1214      0.027      4.480      0.000       0.068       0.175
Nat            0.0458      0.017      2.633      0.008       0.012       0.080
Mar            0.4101      0.029     14.095      0.000       0.353       0.467
L Th          -0.2037      0.023     -8.914      0.000      -0.249      -0.159
Lon            0.1983      0.027      7.402      0.000       0.146       0.251
Ldr            0.0011      0.015      0.072      0.942      -0.028       0.031
Kic            0.5211      0.043     12.031      0.000       0.436       0.606
Jum            0.4473      0.022     19.885      0.000       0.403       0.491
Hea            0.9187      0.023     39.703      0.000       0.873       0.964
Han            0.6642      0.043     15.356      0.000       0.579       0.749
Fre            0.0507      0.029      1.727      0.084      -0.007       0.108
Fla            0.4596      0.019     24.598      0.000       0.423       0.496
Fir            0.5829      0.028     21.080      0.000       0.529       0.637
Fin            0.1702      0.027      6.358      0.000       

In [37]:
reg_multi_CA = sm.OLS(train['CA'], train[pred_vars], hasconst=False).fit()
reg_multi_CA.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     CA   R-squared (uncentered):                   0.990
Model:                            OLS   Adj. R-squared (uncentered):              0.990
Method:                 Least Squares   F-statistic:                          1.384e+05
Date:                Sun, 21 Nov 2021   Prob (F-statistic):                        0.00
Time:                        20:15:56   Log-Likelihood:                     -2.5276e+05
No. Observations:               72375   AIC:                                  5.056e+05
Df Residuals:                   72323   BIC:                                  5.061e+05
Df Model:                          52                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Height        -0.5426      0.003   -158.924      0.000      -0.549      -0.536
Weight         0.2992      0.007     40.468      0.000       0.285       0.314
Age           -0.0985      0.007    -15.134      0.000      -0.111      -0.086
Value       4.326e-07   2.08e-08     20.775      0.000    3.92e-07    4.73e-07
Wage        7.089e-05   5.08e-06     13.953      0.000    6.09e-05    8.09e-05
Wor            0.1565      0.018      8.717      0.000       0.121       0.192
Vis            0.1221      0.016      7.686      0.000       0.091       0.153
Thr            0.2237      0.026      8.525      0.000       0.172       0.275
Tec            0.3504      0.018     19.722      0.000       0.316       0.385
Tea           -0.2643      0.021    -12.299      0.000      -0.306      -0.222
Tck            0.2035      0.015     13.266      0.000       0.173       0.234
Str            0.6024      0.016     37.567      0.000       0.571       0.634
Sta            0.6076      0.013     45.209      0.000       0.581       0.634
TRO            0.0599      0.022      2.745      0.006       0.017       0.103
Ref            0.6526      0.025     25.951      0.000       0.603       0.702
Pun            0.0004      0.021      0.019      0.985      -0.041       0.042
Pos            0.8130      0.016     50.622      0.000       0.782       0.845
Pen            0.1065      0.013      7.998      0.000       0.080       0.133
Pas            0.6690      0.016     41.222      0.000       0.637       0.701
Pac            0.9407      0.022     43.139      0.000       0.898       0.983
1v1            0.2268      0.027      8.546      0.000       0.175       0.279
OtB            0.3141      0.016     19.831      0.000       0.283       0.345
Nat            0.0635      0.010      6.250      0.000       0.044       0.083
Mar            0.6196      0.017     36.431      0.000       0.586       0.653
L Th          -0.1560      0.013    -11.677      0.000      -0.182      -0.130
Lon            0.1290      0.016      8.242      0.000       0.098       0.160
Ldr            0.1272      0.009     14.505      0.000       0.110       0.144
Kic            0.4389      0.025     17.337      0.000       0.389       0.489
Jum            0.9063      0.013     68.924      0.000       0.881       0.932
Hea            0.6921      0.014     51.172      0.000       0.666       0.719
Han            0.7284      0.025     28.811      0.000       0.679       0.778
Fre            0.1765      0.017     10.290      0.000       0.143       0.210
Fla            0.3432      0.011     31.425      0.000       0.322       0.365
Fir            0.4495      0.016     27.813      0.000       0.418       0.481
Fin            0.2693      0.016     17.210      0.000       

In [46]:
from sklearn.neighbors import KNeighborsRegressor

knr_reg_PA = KNeighborsRegressor()
knr_reg_PA.fit(train[pred_vars], train['PA'])

knr_reg_CA = KNeighborsRegressor()
knr_reg_CA.fit(train[pred_vars], train['CA'])

KNeighborsRegressor()

In [44]:
#Starts 7:33 7:44 15 mins
modelsPA = [rf_reg_PA, mlp_reg_PA, kn_reg_PA, reg_multi_PA, svr_reg_PA]

for reg in modelsPA:
    reg_pred = reg.predict(test[pred_vars])

    reg_rmse = metrics.mean_squared_error(test['PA'], reg_pred, squared=False)
    print(reg, "RMSE:", reg_rmse)

RandomForestRegressor() RMSE: 12.119818499869355
MLPRegressor() RMSE: 464.1340717342978
KNeighborsClassifier() RMSE: 22.466858647314517
<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001D9815A2370> RMSE: 13.591567158906548
SVR() RMSE: 20.871684164636093


In [45]:
modelsCA = [rf_reg_CA, mlp_reg_CA, kn_reg_CA, reg_multi_CA, svr_reg_CA]
for reg in modelsCA:
    
    reg_pred = reg.predict(test[pred_vars])

    reg_rmse = metrics.mean_squared_error(test['CA'], reg_pred, squared=False)
    print(reg, "RMSE:", reg_rmse)

RandomForestRegressor() RMSE: 7.046729473366474
MLPRegressor() RMSE: 1028.9799849365702
KNeighborsClassifier() RMSE: 16.91466380003009
<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001D983124610> RMSE: 7.958483468307702
SVR() RMSE: 19.278482160596745


In [47]:
reg_pred = knr_reg_PA.predict(test[pred_vars])

reg_rmse = metrics.mean_squared_error(test['PA'], reg_pred, squared=False)
print(knr_reg_PA, "RMSE:", reg_rmse)

reg_pred = knr_reg_CA.predict(test[pred_vars])

reg_rmse = metrics.mean_squared_error(test['CA'], reg_pred, squared=False)
print(knr_reg_CA, "RMSE:", reg_rmse)

KNeighborsRegressor() RMSE: 15.167702510509724
KNeighborsRegressor() RMSE: 11.101983192733828
